In [1]:
import pandas as pd
import numpy as np
import random as random
random.seed(3)

In [2]:
random.random()

0.23796462709189137

## Plant Location  

#### Manufacturing variable costs

In [3]:
# Import Costs
manvar_costs = pd.read_excel('variable_costs.xlsx', index_col = 0)
manvar_costs

,USA,Germany,Japan,Brazil,India
Variable Costs ($/Unit),,,,,
USA,12,12,12,12,12
Germany,13,13,13,13,13
Japan,10,10,10,10,10
Brazil,8,8,8,8,8
India,5,5,5,5,5


In [4]:
time_cost = pd.read_excel('time_cost.xlsx', index_col = 0)

In [5]:
time_cost

,USA,Germany,Japan,Brazil,India
Variable time (hours),,,,,
USA,0,10,8,2,32
Germany,10,0,28,8,12
Japan,8,28,0,10,14
Brazil,2,8,10,0,18
India,32,12,14,18,0


In [6]:
fresh = 30
fresh_time = time_cost.copy()
for i in fresh_time:
    for j in fresh_time:
        if i!=j:
            fresh_time[i][j] = fresh

In [7]:
fresh_time

,USA,Germany,Japan,Brazil,India
Variable time (hours),,,,,
USA,0,30,30,30,30
Germany,30,0,30,30,30
Japan,30,30,0,30,30
Brazil,30,30,30,0,30
India,30,30,30,30,0


In [8]:
expire_time = time_cost - fresh_time
expire_time

,USA,Germany,Japan,Brazil,India
Variable time (hours),,,,,
USA,0,-20,-22,-28,2
Germany,-20,0,-2,-22,-18
Japan,-22,-2,0,-20,-16
Brazil,-28,-22,-20,0,-12
India,2,-18,-16,-12,0


#### Freight costs

In [9]:
# Import Costs
freight_costs = pd.read_excel('freight_costs.xlsx', index_col = 0)
freight_costs

,USA,Germany,Japan,Brazil,India
Freight Costs ($/Container),,,,,
USA,0,12250,1100,16100,8778
Germany,13335,0,8617,20244,10073
Japan,15400,22750,0,43610,14350
Brazil,16450,22050,28000,0,29750
India,13650,15400,24500,29400,0


#### Variable Costs

In [10]:
# Variable Costs
var_cost = freight_costs/1000 + manvar_costs 

var_cost

,USA,Germany,Japan,Brazil,India
Freight Costs ($/Container),,,,,
USA,12.000,24.25,13.100,28.100,20.778
Germany,26.335,13.00,21.617,33.244,23.073
Japan,25.400,32.75,10.000,53.610,24.350
Brazil,24.450,30.05,36.000,8.000,37.750
India,18.650,20.40,29.500,34.400,5.000


#### Fixed Costs

In [11]:
# Import Costs
fixed_costs = pd.read_excel('fixed_cost.xlsx', index_col = 0)
fixed_costs

,Low,High
USA,6500,9500
Germany,4980,7270
Japan,6230,9100
Brazil,3230,4730
India,2110,6160


#### Plants Capacity

In [12]:
# Two types of plants: Low Capacity and High Capacity Plant
cap = pd.read_excel('capacity.xlsx', index_col = 0)
cap

,Low,High
Capacity (kUnits/month),,
USA,500,1500
Germany,500,1500
Japan,500,1500
Brazil,500,1500
India,500,3000


In [13]:
cap['Low']['USA']

500

#### Demand 

In [14]:
# -- Demand
demand = pd.read_excel('demand_3.xlsx')

In [15]:
demand

,(Units/month),Demand
0,USA,2800000
1,Germany,550000
2,Japan,0
3,Brazil,0
4,India,239000


In [16]:
tot_demand = 0
for i in demand['Demand']:
    tot_demand += i
tot_demand

3589000

In [17]:
demand_center = []
demand_qnty = []
for i, j in zip(demand['(Units/month)'], demand['Demand']):
    print(i)
    demand_center.append(i)
    demand_qnty.append(j)

USA
Germany
Japan
Brazil
India


In [18]:
demand_qnty

[2800000, 550000, 0, 0, 239000]

In [19]:
prdn_center = []
prdn_qnty = []
for i in demand_center:
    for j in ['Low', 'High']:
        prdn_center.append(j+' '+i)
        prdn_qnty.append(cap[j][i])

In [20]:
prdn_center, prdn_qnty

(['Low USA',
  'High USA',
  'Low Germany',
  'High Germany',
  'Low Japan',
  'High Japan',
  'Low Brazil',
  'High Brazil',
  'Low India',
  'High India'],
 [500, 1500, 500, 1500, 500, 1500, 500, 1500, 500, 3000])

In [21]:
loc_seq = random.sample([i for i in range(len(demand_center))], k=len(demand_center))

In [22]:
loc_seq

[4, 1, 3, 2, 0]

In [23]:
# function to select the initial plants and their production
def get_palnts_prod():
    global tot_demand, prdn_center, prdn_qnty
    net_cost = tot_demand
    temp = 0
    t = random.randint(0, 9)
    t=t-4
    while(temp - net_cost<=0):
        temp = 0
        prod_site = []
        prod_qunatity = []
        prod_seq = random.sample([i for i in range(len(prdn_center))], k=len(prdn_center))
        for i in prod_seq:
            if (temp-net_cost<=0):
                prod_site.append(prdn_center[i])
                #temp_prod = random.randint(1, 100)*prdn_qnty[i]*10
                temp_prod = prdn_qnty[i]*1000
                prod_qunatity.append(temp_prod)
                temp += temp_prod
            else:
                #prod_qunatity[-1] =  net_cost - (np.sum(prod_qunatity) - prod_qunatity[-1])
                if t>=0:
                    t=t-1
                    prod_site.append(prdn_center[i])
                    temp_prod = prdn_qnty[i]*1000
                    prod_qunatity.append(temp_prod)
                    print('added extra prdn cntr')
                else:
                    break
    return prod_site, prod_qunatity

def cfactorial(x):
    if x==1:
        return 1
    else:
        return x*cfactorial(x-1)

def get_prod_status(demand, prod_list, prod_quantity, demand_center):
    global time_cost
    # please remove the site with zero balance first
    for i in range(len(prod_list)):
        if prod_quantity[i]<=0:
            prod_list.pop(i)
            prod_quantity.pop(i)
    l_c = []
    l_nc = []
    for i in range(len(prod_list)):
        if prod_quantity[i]>=demand:
            temp = prod_list[i].split()
#             print(time_cost[demand_center][temp[1]], '\n hi here i am')
            if time_cost[demand_center][temp[1]] <= fresh:
                l_c.append(prod_list[i])
        else:
            temp = prod_list[i].split()
            if time_cost[demand_center][temp[1]] <= fresh:
                l_nc.append(prod_list[i])
    return l_c, l_nc

def get_prod_site(demand, prod_list, prod_quantity, demand_center):
    f_l_c, f_l_nc = get_prod_status(demand, prod_list, prod_quantity, demand_center)
    l_c = []
    l_nc = []
    for i in f_l_c:
        l_c.append([i])
    for i in f_l_nc:
        l_nc.append([i])
    t = 0
    while len(l_nc)!=0:
        for i in l_nc:
            t+=1
#             print(t)
            temp_prod_list = prod_list.copy()
            temp_prod_quantity = prod_quantity.copy()
            temp_demand = demand
            for j in i:
                loc = temp_prod_list.index(j)
                temp_prod_list.pop(loc)
                temp_cost = temp_prod_quantity.pop(loc)
                temp_demand = temp_demand - temp_cost
            temp_l_c, temp_l_nc = get_prod_status(temp_demand, temp_prod_list.copy(), temp_prod_quantity.copy(), demand_center)
#             print(l_c, l_nc, 'sd\n\n')
#             print(i, 'i')
#             print(temp_l_c, temp_l_nc, 'cdcsd\n\n')
            for k in temp_l_c:
#                 print(i)
#                 print(i, 'to')
#                 print('temp1\n\n')
                t_0_2 = i.copy()
                t_0_2.append(k)
#                 print(t_0_2, 'final\n')
                l_c.append(t_0_2)
            for l in temp_l_nc:
                t_0_3 = i.copy()
                t_0_3.append(l)
                l_nc.append(t_0_3)
            l_nc.remove(i)
#             print(l_c, l_nc, 'ssadawdd\n\n')
        if t>=10000:
#             print('Caution code not completed')
            break
    return l_c

In [24]:
for i in range(100):
    out = get_palnts_prod()
    print(out)
    print(np.sum(out[1]))

added extra prdn cntr
added extra prdn cntr
added extra prdn cntr
added extra prdn cntr
added extra prdn cntr
(['Low USA', 'High Brazil', 'Low Japan', 'Low India', 'High USA', 'High Japan', 'High Germany', 'Low Germany', 'Low Brazil', 'High India'], [500000, 1500000, 500000, 500000, 1500000, 1500000, 1500000, 500000, 500000, 3000000])
11500000
(['High Germany', 'Low Germany', 'Low Brazil', 'High Japan'], [1500000, 500000, 500000, 1500000])
4000000
(['Low Japan', 'High Brazil', 'Low Brazil', 'High Japan'], [500000, 1500000, 500000, 1500000])
4000000
(['Low Germany', 'High Brazil', 'High Germany', 'High India'], [500000, 1500000, 1500000, 3000000])
6500000
added extra prdn cntr
added extra prdn cntr
added extra prdn cntr
added extra prdn cntr
added extra prdn cntr
(['High India', 'Low Brazil', 'High Germany', 'Low Germany', 'High Japan', 'Low USA', 'Low India', 'High Brazil'], [3000000, 500000, 1500000, 500000, 1500000, 500000, 500000, 1500000])
9500000
added extra prdn cntr
added extra 

In [25]:
dmnd_cntr = []
dmnd_cap = []
for i, j in zip(out[0], out[1]):
    dmnd_cntr.append(i)
    dmnd_cap.append(1000*prdn_qnty[prdn_center.index(i)])

In [26]:
dmnd_cntr, dmnd_cap

(['High USA', 'Low Brazil', 'High India', 'Low Germany'],
 [1500000, 500000, 3000000, 500000])

In [27]:
len(out)

2

In [28]:
out

(['High USA', 'Low Brazil', 'High India', 'Low Germany'],
 [1500000, 500000, 3000000, 500000])

In [29]:
def get_total_cost(loc, prod_site, plant_cap, items):
#     global cap
#     if items>cap[plant_cap][prod_site]*1000:
#         print('plant not having enough items')
#         return None
#     else:
#         return fixed_costs[plant_cap][prod_site] + var_cost[prod_site][loc]*items
#     return fixed_costs[plant_cap][prod_site] + var_cost[prod_site][loc]*items
    return fixed_costs[plant_cap][prod_site]*1000 + var_cost[loc][prod_site]*items

def get_list_cost(loc, plant_list, plant_cap_dict, items):
    cost = 0
    flag = True
    for i in plant_list:
        k = i.split()
        temp_item = plant_cap_dict[i]
        if temp_item > items:
            temp_cost = get_total_cost(loc, k[1], k[0], items)
            cost+=temp_cost
            items = 0
            break
        else:
            items = items - temp_item
            temp_cost = get_total_cost(loc, k[1], k[0], temp_item)
            cost+=temp_cost
    if items>0:
        print("not enough invetory in the list")
        flag = False
    return cost, flag

In [30]:
def get_low_cost_center(loc, all_list, plant_cap_dict_, items):
    low = np.inf
    low_seq = []
    flag = False
    for i in all_list:
        temp_cost, flag = get_list_cost(loc, i, plant_cap_dict_, items)
#         print(i, temp_cost)
        if temp_cost < low:
            low = temp_cost
            low_seq = i
#     print(low, low_seq)
    return low, low_seq, flag

def final_sim():
    global tot_demand
    # get list of production site, list of cap
    plants_seq = get_palnts_prod()
    prdn_cntr = []
    prdn_cap = []
    for i, j in zip(plants_seq[0], plants_seq[1]):
        prdn_cntr.append(i)
        prdn_cap.append(1000*prdn_qnty[prdn_center.index(i)])
    plant_cap_dict_ = {}
    for i, j in zip(prdn_cntr, prdn_cap):
        plant_cap_dict_[i] = j
#     print(prdn_cntr)
#     print(plant_cap_dict_)
    dmnd_cntr = demand_center.copy()
    dmnd_qnty = demand_qnty.copy()
    # select a city
    city_seq = random.sample([i for i in range(len(dmnd_cntr))], k=len(dmnd_cntr))
    print(city_seq, dmnd_cntr, dmnd_qnty)
    # query the list based on time and get new list
    main_output_list = []
    supply_served = 0
    for i in city_seq:
        prdn_cntr_ready = get_prod_site(dmnd_qnty[i], prdn_cntr.copy(), prdn_cap.copy(), dmnd_cntr[i])
        #print(prdn_center)
        nearest_option = get_low_cost_center(dmnd_cntr[i], prdn_cntr_ready, plant_cap_dict_, dmnd_qnty[i])
#         print(nearest_option, '\n')
        if not nearest_option[2]:
            return (False, False)
        else:
            main_output_list.append([nearest_option, dmnd_cntr[i], dmnd_qnty[i]])
            for k in nearest_option[1]:
                if plant_cap_dict_[k]>=dmnd_qnty[i]:
                    print(plant_cap_dict_, 'check1\n\n')
                    print(k)
                    plant_cap_dict_[k] = plant_cap_dict_[k] - dmnd_qnty[i]
                    prdn_cap[prdn_cntr.index(k)] = prdn_cap[prdn_cntr.index(k)] - dmnd_qnty[i]
                    print(plant_cap_dict_, 'check2\n\n')
                else:
                    dmnd_qnty[i] = dmnd_qnty[i] - plant_cap_dict_[k]
                    plant_cap_dict_[k] = 0
                    prdn_cap[prdn_cntr.index(k)] = 0
                    temp_loc = prdn_cntr.index(k)
                    prdn_cntr.pop(temp_loc)
                    prdn_cap.pop(temp_loc)
            supply_served+=dmnd_qnty[i]
            print('total supply given:', supply_served)
            print('total supply remaining:',tot_demand-supply_served)
            print('total capacity left:\n', plant_cap_dict_)
        # get the cheapest site
        # put the order -> remove the capcoty from the "list of cap"
        # if production site is exhausted then remove it
        # Go to next city
    return (True, main_output_list)

In [31]:
low = np.inf
best_ = []
for j in range(1000):
    try:
        y = final_sim()
        if y[0]:
            temp_cost = 0
            for i in y[1]:
                temp_cost+=i[0][0] 
            if temp_cost<low:
                low = temp_cost
                best_ = y
    except:
        pass
#     y = final_sim()
#     if y[0]:
#         temp_cost = 0
#         for i in y[1]:
#             temp_cost+=i[0][0] 
#         if temp_cost<low:
#             low = temp_cost
#             best_ = y
    print(low)

added extra prdn cntr
[2, 4, 3, 1, 0] ['USA', 'Germany', 'Japan', 'Brazil', 'India'] [2800000, 550000, 0, 0, 239000]
{'High Japan': 1500000, 'Low Germany': 500000, 'Low Brazil': 500000, 'High Brazil': 1500000, 'High Germany': 1500000} check1


Low Brazil
{'High Japan': 1500000, 'Low Germany': 500000, 'Low Brazil': 500000, 'High Brazil': 1500000, 'High Germany': 1500000} check2


total supply given: 0
total supply remaining: 3589000
total capacity left:
 {'High Japan': 1500000, 'Low Germany': 500000, 'Low Brazil': 500000, 'High Brazil': 1500000, 'High Germany': 1500000}
{'High Japan': 1500000, 'Low Germany': 500000, 'Low Brazil': 500000, 'High Brazil': 1500000, 'High Germany': 1500000} check1


Low Germany
{'High Japan': 1500000, 'Low Germany': 261000, 'Low Brazil': 500000, 'High Brazil': 1500000, 'High Germany': 1500000} check2


total supply given: 239000
total supply remaining: 3350000
total capacity left:
 {'High Japan': 1500000, 'Low Germany': 261000, 'Low Brazil': 500000, 'High Br

{'High Japan': 1500000, 'Low India': 261000, 'Low Germany': 500000, 'Low Japan': 500000, 'Low Brazil': 500000, 'High India': 3000000, 'High Germany': 950000, 'High Brazil': 1500000, 'Low USA': 500000, 'High USA': 0} check1


High Brazil
{'High Japan': 1500000, 'Low India': 261000, 'Low Germany': 500000, 'Low Japan': 500000, 'Low Brazil': 500000, 'High India': 3000000, 'High Germany': 950000, 'High Brazil': 200000, 'Low USA': 500000, 'High USA': 0} check2


total supply given: 2089000
total supply remaining: 1500000
total capacity left:
 {'High Japan': 1500000, 'Low India': 261000, 'Low Germany': 500000, 'Low Japan': 500000, 'Low Brazil': 500000, 'High India': 3000000, 'High Germany': 950000, 'High Brazil': 200000, 'Low USA': 500000, 'High USA': 0}
85960000.0
[3, 4, 2, 1, 0] ['USA', 'Germany', 'Japan', 'Brazil', 'India'] [2800000, 550000, 0, 0, 239000]
{'Low Brazil': 500000, 'High India': 3000000, 'High Germany': 1500000} check1


Low Brazil
{'Low Brazil': 500000, 'High India': 3000000,

{'Low Brazil': 500000, 'Low Japan': 500000, 'High Japan': 1500000, 'High Germany': 950000, 'High Brazil': 1500000, 'Low India': 261000, 'High USA': 0, 'Low USA': 500000, 'High India': 3000000} check1


High Brazil
{'Low Brazil': 500000, 'Low Japan': 500000, 'High Japan': 1500000, 'High Germany': 950000, 'High Brazil': 200000, 'Low India': 261000, 'High USA': 0, 'Low USA': 500000, 'High India': 3000000} check2


total supply given: 2089000
total supply remaining: 1500000
total capacity left:
 {'Low Brazil': 500000, 'Low Japan': 500000, 'High Japan': 1500000, 'High Germany': 950000, 'High Brazil': 200000, 'Low India': 261000, 'High USA': 0, 'Low USA': 500000, 'High India': 3000000}
85960000.0
added extra prdn cntr
added extra prdn cntr
[1, 4, 0, 3, 2] ['USA', 'Germany', 'Japan', 'Brazil', 'India'] [2800000, 550000, 0, 0, 239000]
{'High India': 3000000, 'Low India': 500000, 'High Germany': 1500000, 'High Brazil': 1500000, 'Low USA': 500000} check1


High Germany
{'High India': 3000000, 'L

{'Low Germany': 500000, 'High Japan': 1500000, 'High Brazil': 1500000, 'High USA': 0, 'Low Japan': 500000, 'High Germany': 1500000, 'Low India': 500000, 'Low USA': 500000, 'Low Brazil': 500000} check1


High Brazil
{'Low Germany': 500000, 'High Japan': 1500000, 'High Brazil': 200000, 'High USA': 0, 'Low Japan': 500000, 'High Germany': 1500000, 'Low India': 500000, 'Low USA': 500000, 'Low Brazil': 500000} check2


total supply given: 1300000
total supply remaining: 2289000
total capacity left:
 {'Low Germany': 500000, 'High Japan': 1500000, 'High Brazil': 200000, 'High USA': 0, 'Low Japan': 500000, 'High Germany': 1500000, 'Low India': 500000, 'Low USA': 500000, 'Low Brazil': 500000}
{'Low Germany': 500000, 'High Japan': 1500000, 'High Brazil': 200000, 'High USA': 0, 'Low Japan': 500000, 'High Germany': 1500000, 'Low India': 500000, 'Low USA': 500000, 'Low Brazil': 500000} check1


Low India
{'Low Germany': 500000, 'High Japan': 1500000, 'High Brazil': 200000, 'High USA': 0, 'Low Japan'

{'High Brazil': 1500000, 'High Germany': 1500000, 'Low USA': 500000, 'Low Germany': 500000, 'High USA': 0, 'High Japan': 1500000, 'Low Japan': 500000, 'Low India': 261000} check1


High Brazil
{'High Brazil': 200000, 'High Germany': 1500000, 'Low USA': 500000, 'Low Germany': 500000, 'High USA': 0, 'High Japan': 1500000, 'Low Japan': 500000, 'Low India': 261000} check2


total supply given: 1539000
total supply remaining: 2050000
total capacity left:
 {'High Brazil': 200000, 'High Germany': 1500000, 'Low USA': 500000, 'Low Germany': 500000, 'High USA': 0, 'High Japan': 1500000, 'Low Japan': 500000, 'Low India': 261000}
{'High Brazil': 200000, 'High Germany': 1500000, 'Low USA': 500000, 'Low Germany': 500000, 'High USA': 0, 'High Japan': 1500000, 'Low Japan': 500000, 'Low India': 261000} check1


High Germany
{'High Brazil': 200000, 'High Germany': 950000, 'Low USA': 500000, 'Low Germany': 500000, 'High USA': 0, 'High Japan': 1500000, 'Low Japan': 500000, 'Low India': 261000} check2


to

...

{'Low India': 500000, 'Low Japan': 500000, 'High Japan': 1500000, 'Low Germany': 500000, 'High USA': 0, 'Low Brazil': 0, 'High India': 3000000, 'Low USA': 0, 'High Germany': 950000} check1


Low Germany
{'Low India': 500000, 'Low Japan': 500000, 'High Japan': 1500000, 'Low Germany': 200000, 'High USA': 0, 'Low Brazil': 0, 'High India': 3000000, 'Low USA': 0, 'High Germany': 950000} check2


total supply given: 850000
total supply remaining: 2739000
total capacity left:
 {'Low India': 500000, 'Low Japan': 500000, 'High Japan': 1500000, 'Low Germany': 200000, 'High USA': 0, 'Low Brazil': 0, 'High India': 3000000, 'Low USA': 0, 'High Germany': 950000}
{'Low India': 500000, 'Low Japan': 500000, 'High Japan': 1500000, 'Low Germany': 200000, 'High USA': 0, 'Low Brazil': 0, 'High India': 3000000, 'Low USA': 0, 'High Germany': 950000} check1


Low India
{'Low India': 500000, 'Low Japan': 500000, 'High Japan': 1500000, 'Low Germany': 200000, 'High USA': 0, 'Low Brazil': 0, 'High India': 3000000

{'Low USA': 500000, 'High Brazil': 1500000, 'Low Japan': 500000, 'High USA': 0, 'High Germany': 950000, 'Low Germany': 500000, 'Low Brazil': 500000, 'High Japan': 1500000, 'High India': 3000000, 'Low India': 261000} check1


High Brazil
{'Low USA': 500000, 'High Brazil': 200000, 'Low Japan': 500000, 'High USA': 0, 'High Germany': 950000, 'Low Germany': 500000, 'Low Brazil': 500000, 'High Japan': 1500000, 'High India': 3000000, 'Low India': 261000} check2


total supply given: 2089000
total supply remaining: 1500000
total capacity left:
 {'Low USA': 500000, 'High Brazil': 200000, 'Low Japan': 500000, 'High USA': 0, 'High Germany': 950000, 'Low Germany': 500000, 'Low Brazil': 500000, 'High Japan': 1500000, 'High India': 3000000, 'Low India': 261000}
{'Low USA': 500000, 'High Brazil': 200000, 'Low Japan': 500000, 'High USA': 0, 'High Germany': 950000, 'Low Germany': 500000, 'Low Brazil': 500000, 'High Japan': 1500000, 'High India': 3000000, 'Low India': 261000} check1


Low India
{'Low US

{'High Germany': 950000, 'Low USA': 500000, 'Low Japan': 500000, 'Low India': 261000, 'High Japan': 1500000, 'Low Germany': 500000, 'High USA': 0, 'High India': 3000000, 'Low Brazil': 500000, 'High Brazil': 1500000} check1


High Brazil
{'High Germany': 950000, 'Low USA': 500000, 'Low Japan': 500000, 'Low India': 261000, 'High Japan': 1500000, 'Low Germany': 500000, 'High USA': 0, 'High India': 3000000, 'Low Brazil': 500000, 'High Brazil': 200000} check2


total supply given: 2089000
total supply remaining: 1500000
total capacity left:
 {'High Germany': 950000, 'Low USA': 500000, 'Low Japan': 500000, 'Low India': 261000, 'High Japan': 1500000, 'Low Germany': 500000, 'High USA': 0, 'High India': 3000000, 'Low Brazil': 500000, 'High Brazil': 200000}
85960000.0
[3, 2, 4, 1, 0] ['USA', 'Germany', 'Japan', 'Brazil', 'India'] [2800000, 550000, 0, 0, 239000]
{'High Germany': 1500000, 'Low Germany': 500000, 'Low USA': 500000, 'High India': 3000000} check1


Low Germany
{'High Germany': 1500000

{'High USA': 0, 'Low Brazil': 0, 'High India': 3000000, 'Low Japan': 500000, 'High Japan': 1500000, 'High Germany': 950000, 'Low USA': 0, 'Low Germany': 500000} check1


Low Germany
{'High USA': 0, 'Low Brazil': 0, 'High India': 3000000, 'Low Japan': 500000, 'High Japan': 1500000, 'High Germany': 950000, 'Low USA': 0, 'Low Germany': 200000} check2


total supply given: 850000
total supply remaining: 2739000
total capacity left:
 {'High USA': 0, 'Low Brazil': 0, 'High India': 3000000, 'Low Japan': 500000, 'High Japan': 1500000, 'High Germany': 950000, 'Low USA': 0, 'Low Germany': 200000}
{'High USA': 0, 'Low Brazil': 0, 'High India': 3000000, 'Low Japan': 500000, 'High Japan': 1500000, 'High Germany': 950000, 'Low USA': 0, 'Low Germany': 200000} check1


Low Germany
{'High USA': 0, 'Low Brazil': 0, 'High India': 3000000, 'Low Japan': 500000, 'High Japan': 1500000, 'High Germany': 950000, 'Low USA': 0, 'Low Germany': 200000} check2


total supply given: 850000
total supply remaining: 273

{'High Brazil': 1500000, 'High India': 3000000, 'High Japan': 1500000, 'Low Germany': 500000, 'Low Japan': 500000, 'Low India': 500000, 'High Germany': 1500000, 'High USA': 0} check1


High Brazil
{'High Brazil': 200000, 'High India': 3000000, 'High Japan': 1500000, 'Low Germany': 500000, 'Low Japan': 500000, 'Low India': 500000, 'High Germany': 1500000, 'High USA': 0} check2


total supply given: 1300000
total supply remaining: 2289000
total capacity left:
 {'High Brazil': 200000, 'High India': 3000000, 'High Japan': 1500000, 'Low Germany': 500000, 'Low Japan': 500000, 'Low India': 500000, 'High Germany': 1500000, 'High USA': 0}
{'High Brazil': 200000, 'High India': 3000000, 'High Japan': 1500000, 'Low Germany': 500000, 'Low Japan': 500000, 'Low India': 500000, 'High Germany': 1500000, 'High USA': 0} check1


High Germany
{'High Brazil': 200000, 'High India': 3000000, 'High Japan': 1500000, 'Low Germany': 500000, 'Low Japan': 500000, 'Low India': 500000, 'High Germany': 950000, 'High 

In [32]:
best_

(True,
 [[(14420000.0, ['High Germany'], True), 'Germany', 550000],
  [(2110000.0, ['Low India'], True), 'Japan', 0],
  [(64015000.0, ['High USA', 'High Brazil'], True), 'USA', 2800000],
  [(3305000.0, ['Low India'], True), 'India', 239000],
  [(2110000.0, ['Low India'], True), 'Brazil', 0]])

In [33]:
temp_cost = 0
for i in best_[1]:
    temp_cost+=i[0][0] 
print(temp_cost)

85960000.0
